Ruggiero Seccia

La Sapienza University of Rome

Email: ruggiero.seccia@uniroma1.it

Phone: +39 3318606535

# Standard Nurses Rostering Problem (V2)


This notebook implements a model of the Nurses rostering problem. With respect to V1 this notebook include the possibility that __the number of nurses is not enough to guarantee the satisfaction of the working condition regulations within the ward__. While in the previous case (V1) the problem was not feasible every time $N$ was not big enough, now the problem is ensured to be feasible by adding the slack variables $\alpha_{st}$.



## importing the packages

In [20]:
import numpy as np
try:
    from docplex.mp.model import Model
except:
    !pip install docplex
from docplex.mp.model import Model

import pandas as pd

try:
    import ipywidgets
except:
    !pip install ipywidgets
from ipywidgets import interact
import ipywidgets as widgets
import time

## Parameters specification

Let us consider a department in a hospital with a given number of nurses $N$. We want to organize their shifts for the next $T$ days, e.g. $T=7$ one week or $T=30$ next month,  and for all the followings so to minimize the effort required by the staff to satisfy the demand. By contract, each nurse $i$ has to work $H_i$ hours over the time horizon $T$ (e.g. each nurse must work at least 36 hours per week, $H=36$ and $T=7$). If the $i$th nurse works for a number of hours higher than $H_i$, then it is counted as extra work and then paid more by the healthcare structure.  However, nurses cannot work for more than $H^{\max}$ hours per week, so to avoid burning out. Each day three shifts need to be covered by the nurses: morning, afternoon and night. Each shift $s$ requires $R_s$ nurses and lasts $h_s$ hours. Each nurse cannot cover more than one shift per day. Moreover, we have the further constraint that if a nurse covers a night shift then they need to rest and cannot work the following day.

Let us  consider the parameter $p_i$ which brings information about the previous period. Namely, $p_i$ is a boolean parameter such that 
\begin{equation*}
    p_i=
    \begin{cases}
    1  \qquad \text{if the } i \text{th nurse worked on the last day of the previous period} \\
    0 \qquad \text{otherwise.}
    \end{cases}
\end{equation*}


In [28]:
# number of nurses
N = 15
nurses = ['Nurse_' +str(n) for n in range(N)]
# periods to schedule
T = 7
days = ['Day_' +str(t) for t in range(T)]
# shifts
S = ['Morning', 'Afternoon', 'Night']


# standard number of hours by contract per nurse (e.g. 6 hours per day, 36 per week)
H_base = 36*np.floor(T/7) + 6*np.mod(T,7)
H = {n:j for n in nurses for j in [H_base]*len(nurses)}

# maximum number of hours a nurse can work including the extra shifts (e.g. 10 hours per day, 60 per week)
H_max = 60*np.floor(T/7) + 10*np.mod(T,7)
H_MAX = {n:j for n in nurses for j in [H_max]*len(nurses)}

# update some nurses values
# H['Nurse_1'] =20

# number of nurses required per shift
R = {'Morning' : 5,
     'Afternoon' : 4,
     'Night' : 3}
# duration of each shift
h = {'Morning' : 7,
     'Afternoon' : 8,
     'Night' : 9}

# list of nurses that on the last day of the previous period covered  the night shift
p_list= ['Nurse_0']
# dictionary with the values of p per each nurse
p = {n:0 for n in nurses}
# update the dictionary with p_list
for pp in p_list:
    p[pp]=1


## Optimization model 

To formulate this optimization problem, let us introduce the binary variable $x_{ist}\in\{0,1\}$ such that 
\begin{equation*}
    x_{ist}=
    \begin{cases}
    1  \qquad \text{if nurse } i \text{th covers shift } s \text{th on day } t\text{th} \\
    0 \qquad \text{otherwise}
    \end{cases}
\end{equation*}

Moreover we define the variable $\alpha_{st} \in R $ which represents the number of nurses that are missing to satisfy the minimum demand of the $s$th shift on the $t$th department.

We want to find the optimal schedule $x^\star$ such that the number of hours worked by nurses is minimized and all the department's constraints are satisfied. Of course, we are also interested in minimizing the number of shifts that cannot be covered by the nurses ($\sum_{st}\alpha_st$)

In [29]:
mdl = Model('Scheduling')

# create the variables
idx_x = [(i,s,t) for i in nurses for s in S for t in days]
x = mdl.binary_var_dict(idx_x)
idx_alpha = [(s,t) for s in S for t in days]
alpha = mdl.continuous_var_dict(idx_alpha,lb = 0)


### Objective function
The objective function is asking to minimize the overall number of hours worked by all nurses within the period under consideration while reducing the most the number of hours not covered by nurses

\begin{equation*}
\begin{aligned}
& \underset{x_{ist}\in\{0,1\},\alpha_{st}\in R}{\text{min}}
& & \sum_{i=1}^N\sum_{s=1}^3\sum_{t=1}^T x_{ist}h_s +
  \rho \sum_{s=1}^3\sum_{t=1}^T\alpha_{st}\\
\end{aligned}
\end{equation*}

with $\rho>\max_s h_s$. Note that, even if $\alpha$ represents a discrete quantity, it is modeled as a continuous variable since at the optimum it will achieve only integer values.

In [30]:
# objective function definition
mdl.minimize(mdl.sum(x[i,s,t]*h[s] for i in nurses for s in S for t in days)
             +(max(h.values())+1)*mdl.sum(alpha[s,t] for s in S for t in days))

### Constraints
- Each person cannot cover more than one shift in the same day. 
$$ \sum_{s=1}^3x_{ist}\leq 1 \qquad \forall i=1,...,N \;t=1,...,T \label{eq: 21}$$

- The number of personnel per each shift in each day is satisfied. 
$$\sum_{i=1}^N x_{ist} +\alpha_{st} \geq R_{s} \qquad \forall s=1,...,3 \; t=1,...,T  \label{eq: 22} $$

- Each nurse works at minimum the number of hours required by contract. 
$$ \sum_{s=1}^3\sum_{t=1}^T x_{ist}h_s\geq H_i \qquad \forall i=1,...,N \label{eq: 23}$$
- but at most $H^{\max}$ hours
$$ \sum_{s=1}^3\sum_{t=1}^T x_{ist}h_s\leq H^{\max} \qquad \forall i=1,...,N $$

- If a nurse covers a night shift, then the next day they cannot work;
$$ x_{i3t}+\sum_{s=1}^3 x_{ist+1}\leq 1 \qquad \forall i=1,...N \; t=1,...,T-1\label{eq: 24}$$

- Each nurse cannot work on the first day of the new period if they worked on the last day of the previous period. 
$$ \sum_{s=1}^3 x_{is1}\leq (1-p_i) \qquad \forall i=1,...,N  \label{eq: 25}$$


In [31]:
mdl.add_constraints(mdl.sum(x[i,s,t] for s in S) <= 1 for i in nurses for t in days);

mdl.add_constraints(mdl.sum(x[i,s,t] for i in nurses)+alpha[s,t]== R[s]  for s in S for t in days);

mdl.add_constraints(mdl.sum(x[i,s,t]*h[s] for s in S for t in days) >= H[i] for i in nurses );

mdl.add_constraints( x[i,S[-1],t] + mdl.sum(x[i,s,days[j+1]] for s in S )<= 1 for i in nurses for j,t in enumerate(days[:-1]) );

mdl.add_constraints(mdl.sum(x[i,s,days[0]] for s in S ) <= (1-p[i]) for i in nurses );

### Defining KPI

In [32]:
mdl.add_kpi(mdl.sum(R[s]- mdl.sum(x[i,s,t] for i in nurses)>=1 for s in S for t in days), '# of shifts not completely covered');
mdl.add_kpi(mdl.max(alpha[s,t] for s in S for t in days), 'Maximum # missing nurses in one shift')
mdl.add_kpi(mdl.sum(alpha[s,t] for s in S for t in days), 'Sum of missing nurses in all the shifts')

mdl.add_kpi(mdl.max(mdl.sum(x[i,s,t]*h[s] for s in S for t in days)for i in nurses), 'Maximum # hours worked')
mdl.add_kpi(mdl.min(mdl.sum(x[i,s,t]*h[s] for s in S for t in days)for i in nurses), 'Minimum # hours worked')

DecisionKPI(name=Minimum # hours worked,expr=min(7x1+7x2+7x3+7x4+7x5+7x6+7x7+8x8+8x9+8x10+8x11+8x12+8x13+8x14..)

### Solve the problem

In [33]:
mdl.print_information()
mdl.solve()
mdl.solution.solve_details

Model: Scheduling
 - number of variables: 387
   - binary=336, integer=0, continuous=51
 - number of constraints: 297
   - linear=276, equiv=21
 - parameters: defaults
 - problem type is: MILP


docplex.mp.SolveDetails(time=0.109,status='integer optimal solution')

In [34]:
status = mdl.solve_details.status == 'integer optimal solution'
status

True

In [36]:
mdl.report()

* model Scheduling solved with objective = 658.000
*  KPI: # of shifts not completely covered      = 0.000
*  KPI: Maximum # missing nurses in one shift   = 0.000
*  KPI: Sum of missing nurses in all the shifts = 0.000
*  KPI: Maximum # hours worked                  = 55.000
*  KPI: Minimum # hours worked                  = 36.000


## Analysing the solution

Below we provide simple tools to analyse the solution obtained

In [37]:
def x_star_to_pandas(x):
    '''
    takes in input the solution of the optimization problem as a dictionary 
    returns the solution as a dataframe 
    '''
    sol = pd.DataFrame(columns = ['Nurse', 'Shift', 'Day'])
    k = 0
    for key, value in x.items():
        if value>0:
            sol.loc[k] =np.array([i for i in key])
            k+=1
    return sol

In [38]:
# transform the solution into a dataframe
x_star_dict =mdl.solution.get_value_dict(x)
sol_x = x_star_to_pandas(x_star_dict)
sol_x.head()

,Nurse,Shift,Day
0,Nurse_0,Morning,Day_1
1,Nurse_0,Morning,Day_5
2,Nurse_0,Afternoon,Day_2
3,Nurse_0,Afternoon,Day_3
4,Nurse_0,Afternoon,Day_4


### Is the number of nurses $N$ enough to satisfy the demand?

In [39]:
alpha_star_dict =mdl.solution.get_value_dict(alpha)
if max(list(alpha_star_dict.values()))==0:
    print('YES! The number of nurses is enough to satisfy the demand')
else:
    print('NO! The number of nurses is not enough to satisfy the demand. alpha_star= ',max(list(alpha_star_dict.values())))


YES! The number of nurses is enough to satisfy the demand


In [40]:
list(alpha_star_dict.values())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

### How many hours does each nurse work over the period?

In [41]:
worked_hours = {n:0 for n in nurses}

for i,j in sol_x.iterrows():
    worked_hours[j['Nurse']]+=h[j['Shift']]
worked_hours

{'Nurse_0': 47,
 'Nurse_1': 54,
 'Nurse_2': 54,
 'Nurse_3': 46,
 'Nurse_4': 46,
 'Nurse_5': 40,
 'Nurse_6': 36,
 'Nurse_7': 36,
 'Nurse_8': 37,
 'Nurse_9': 41,
 'Nurse_10': 37,
 'Nurse_11': 44,
 'Nurse_12': 40,
 'Nurse_13': 45,
 'Nurse_14': 55}

### Average of hours worked by day

In [28]:
for i, j in worked_hours.items():
    print(i,':',j/T)

Nurse_0 : 5.714285714285714
Nurse_1 : 6.571428571428571
Nurse_2 : 7.714285714285714
Nurse_3 : 5.142857142857143
Nurse_4 : 7.428571428571429
Nurse_5 : 7.142857142857143
Nurse_6 : 5.571428571428571
Nurse_7 : 7.428571428571429
Nurse_8 : 5.857142857142857
Nurse_9 : 5.857142857142857
Nurse_10 : 6.571428571428571
Nurse_11 : 5.714285714285714
Nurse_12 : 5.857142857142857
Nurse_13 : 5.857142857142857
Nurse_14 : 5.571428571428571


### Visualization tool

Below we provide a tool to check the schedule. 

In [43]:
# remove warning from pandas (in the viz_tool it does what we need)
import warnings
warnings.simplefilter(action='ignore')

In [44]:

def viz_tool(nurse,shift,day):
    '''
    interactive function to extract the information required:
    if a value is 'All' then it returns all the values for that specific feature
    '''
    global nurses,S,days
    
    if nurse == 'All':
        df_tmp = sol_x[(sol_x['Nurse'].isin(nurses))]
    else:
        df_tmp = sol_x[(sol_x['Nurse']==nurse)]

    if shift == 'All':
        df_tmp = df_tmp[(sol_x['Shift'].isin(S))]
    else:
        df_tmp = df_tmp[(sol_x['Shift']==shift)]

    if day == 'All':
        df_tmp = df_tmp[(sol_x['Day'].isin(days))]    
    else:
        df_tmp = df_tmp[(sol_x['Day']==day)]

    print(df_tmp)

interact(viz_tool, nurse = widgets.Dropdown(value="All",placeholder='Type something', options=nurses+['All']),
              shift=widgets.Dropdown(value='All',placeholder='Type something', options=S+['All']),
              day = widgets.Dropdown(value="All",placeholder='Type something', options=days+['All'])
        );



interactive(children=(Dropdown(description='nurse', index=15, options=('Nurse_0', 'Nurse_1', 'Nurse_2', 'Nurse…

## Minimizing the worst case scenario

 We want to modify the objective function so to minimize the worst case scenario, i.e. minimize the maximum number of hours done by a nurse. It can be easily accomplished by introducing the further continuous variable $y\in R$, rewriting the objective function as
\begin{equation*}
    \underset{x_{ist}\in\{0,1\},y\in R,\alpha_{st}\in R}{\text{min}} \quad  y +\rho \sum_{s=1}^3\sum_{t=1}^T\alpha_{st}
\end{equation*}

and introducing the set of constraints:
\begin{equation*}
    y\geq \sum_{s=1}^3\sum_{t=1}^T x_{ist}h_s \qquad \forall i=1,...,N
\end{equation*}

In [45]:
# create the new variable for minimizing the worst case scenario
y = mdl.continuous_var()

# modify the objective function
mdl.minimize(y
             +(max(h.values())+1)*mdl.sum(alpha[s,t] for s in S for t in days))

# add the linear constraint
mdl.add_constraints( y>= mdl.sum( x[i,s,t]*h[s] for s in S for t in days) for i in nurses);

In [46]:
mdl.print_information()
mdl.solve()
mdl.solution.solve_details

Model: Scheduling
 - number of variables: 388
   - binary=336, integer=0, continuous=52
 - number of constraints: 312
   - linear=291, equiv=21
 - parameters: defaults
 - problem type is: MILP


docplex.mp.SolveDetails(time=1.391,status='integer optimal solution')

In [47]:
mdl.report()

* model Scheduling solved with objective = 45.000
*  KPI: # of shifts not completely covered      = 0.000
*  KPI: Maximum # missing nurses in one shift   = 0.000
*  KPI: Sum of missing nurses in all the shifts = 0.000
*  KPI: Maximum # hours worked                  = 45.000
*  KPI: Minimum # hours worked                  = 41.000


## Comparing the new solution with the old one

In [49]:
# transform the solution into a dataframe
x_star_dict =mdl.solution.get_value_dict(x)
sol_x = x_star_to_pandas(x_star_dict)
sol_x.head()

,Nurse,Shift,Day
0,Nurse_0,Morning,Day_2
1,Nurse_0,Morning,Day_3
2,Nurse_0,Morning,Day_4
3,Nurse_0,Morning,Day_6
4,Nurse_0,Afternoon,Day_1


### Compute number of hours worked

In [50]:
# number of hours worked by each nurse 
worked_hours_wc = {n:0 for n in nurses}

for i,j in sol_x.iterrows():
    worked_hours_wc[j['Nurse']]+=h[j['Shift']]


#### Comparison with the previous solution

Hours are more equally distributed!

In [51]:
print("{0:<10s} {1:<10s} {2:<10s}".format("Nurse","Old value", "New value") )
for i in worked_hours.keys():
    print("{0:<10s} {1:<10.0f} {2:<10.0f}".format(i+":",worked_hours[i],worked_hours_wc[i]) )

Nurse      Old value  New value 
Nurse_0:   47         44        
Nurse_1:   54         43        
Nurse_2:   54         41        
Nurse_3:   46         45        
Nurse_4:   46         45        
Nurse_5:   40         45        
Nurse_6:   36         43        
Nurse_7:   36         42        
Nurse_8:   37         45        
Nurse_9:   41         45        
Nurse_10:  37         45        
Nurse_11:  44         45        
Nurse_12:  40         45        
Nurse_13:  45         42        
Nurse_14:  55         43        


### Maximum number of hours worked:

In [52]:
print('Difference on maximum differenceon number of hours worked by each nurse:')
print("Old value: {0:>2.0f} New value: {1:>2.0f}".format(max(worked_hours.values())-min(worked_hours.values()),
                                                         max(worked_hours_wc.values())-min(worked_hours_wc.values()))) 


Difference on maximum differenceon number of hours worked by each nurse:
Old value: 19 New value:  4


In [53]:
interact(viz_tool, nurse = widgets.Dropdown(value="All",placeholder='Type something', options=nurses+['All']),
              shift=widgets.Dropdown(value='All',placeholder='Type something', options=S+['All']),
              day = widgets.Dropdown(value="All",placeholder='Type something', options=days+['All']),
        );



interactive(children=(Dropdown(description='nurse', index=15, options=('Nurse_0', 'Nurse_1', 'Nurse_2', 'Nurse…